In [1]:
import json

from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings

In [2]:
embedding_function = SentenceTransformerEmbeddings(model_name="BAAI/bge-m3")

/home/truberov/Documents/GitHub/tink-green-hack/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/truberov/Documents/GitHub/tink-green-hack/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
from langchain_community.document_loaders import JSONLoader


def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["url"] = record.get("url")
    metadata["title"] = record.get("title")

    return metadata


loader = JSONLoader(
    file_path='./dataset_unique_epta.json',
    jq_schema='.data[]',
    text_content=False,
    content_key='description',
    metadata_func=metadata_func,
)

docs = loader.load()

for doc in docs:
    doc.page_content = f"{doc.metadata.get('title')}\n{doc.page_content}"

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=500, chunk_overlap=0, keep_separator=True, is_separator_regex=True, separators=[r'(?s)[А-ЯЁA-Z].*?\?']
# )
# doc_splits = text_splitter.split_documents(docs)

In [6]:
len(docs)

2880

In [7]:
import uuid
import httpx
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryByteStore

In [8]:
API_KEY = ''

proxies = {
    'http://': '',
    'https://': ''
}

llm = ChatOpenAI(api_key=API_KEY, model='gpt-3.5-turbo', http_client=httpx.Client(proxies=proxies))

In [12]:
chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("Ты эксперт в суммаризации документов для векторного поиска. Суммаризуй следующий документ, не употребляй фразы по типу документ рассказывает, тебе нужно передать смысл документа, сохранив ключевые слова и факты, итоговый документ должен получится меньше исходного:\n\n{doc}")
    | llm
    | StrOutputParser()
)

In [13]:
summaries = chain.batch(docs, {"max_concurrency": 2})

In [18]:
import json

with open('save_summaries.json', 'w') as f:
    f.write(json.dumps(summaries, ensure_ascii=False))

In [19]:
# The vectorstore to use to index the child chunks
vectorstore = Chroma(collection_name="summaries", embedding_function=embedding_function, persist_directory='./chroma_test_summary')
# The storage layer for the parent documents
store = InMemoryByteStore()
id_key = "doc_id"
# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)
doc_ids = [str(uuid.uuid4()) for _ in docs]

In [20]:
summary_docs = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(summaries)
]

In [21]:
retriever.vectorstore.add_documents(summary_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [38]:
retriever.search_kwargs = {"k": 20}

In [39]:
def check_docs_correctness(true_context, _predicted_contexts):
    for predicted_context in _predicted_contexts:
        if true_context in predicted_context:
            return True
    return False

In [41]:
import pandas as pd

df = pd.read_json('./test_data_retriever.json')
correct_in_top5 = 0
correct_present = 0
total_tests = 0

for index, row in df.iterrows():
    context = row["description"]
    questions = row["questions"]
    total_tests += len(questions)

    for question in questions:
        _q = question["question"]
        predicted_contexts_data = retriever.invoke(_q)
        predicted_contexts = [doc.page_content for doc in predicted_contexts_data]
        
        if check_docs_correctness(context, predicted_contexts):
            correct_present += 1
            
            if check_docs_correctness(context, predicted_contexts[:5]):
                correct_in_top5 += 1


top5_percentage = correct_in_top5 / total_tests * 100
present_percentage = correct_present / total_tests * 100

print(f"Верный ответ присутствует в топ-20 в {present_percentage:.2f}% тестов")
print(f"Верный ответ в топ-5 в {top5_percentage:.2f}% тестов")

Верный ответ присутствует в топ-20 в 77.78% тестов
Верный ответ в топ-5 в 61.11% тестов


In [42]:
retriever.invoke("Как добавить новый магазин в личный кабинет руководителя?")

[Document(page_content='Как дать менеджеру доступ к нескольким магазинам?\nЭто можно сделать в личном кабинете руководителя двумя способами. В разделе «Магазины». Выберите магазин из списка или найдите его по названию или адресу и нажмите «Добавить менеджера». Выберите менеджера из выпадающего списка и нажмите «Добавить». В разделе «Менеджеры». Найдите сотрудника по ФИО или магазину, к которому он привязан. Кликните на имя сотрудника и в блоке «Магазины» нажмите «Добавить». Введите название магазина или выберите подходящий из выпадающего списка, снова нажмите «Добавить». К одному магазину можно прикрепить несколько менеджеров. В таком случае каждый сотрудник будет входить в систему под своими данными. Двум менеджерам не получится одновременно работать в одном личном кабинете. Для этого выберите один магазин или несколько в одноименном разделе в личном кабинете руководителя. Здесь же можно посмотреть всех агентов, у которых есть доступы к нему. Как добавить агента в личном кабинете', me